In [ ]:
import os

import cv2
import random
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from PIL import Image
from imgaug import augmenters as iaa

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.utils import *
from keras.callbacks import *

DIRECTORY = "images/Images/"

breeds = os.listdir(DIRECTORY) #To get the list of all the breeds

BREEDS = breeds

IMG_SIZE = 224

In [ ]:
label_maps = {}
label_maps_reverse = {}
for label, number in enumerate(breeds):
    label_maps.update({label: number})
    label_maps_reverse.update({number : label})

In [ ]:
training_data = []

def create_training_data():
    for breed in BREEDS :
        path = os.path.join(DIRECTORY, breed)
        label_to_number = BREEDS.index(breed)
        for image in os.listdir(path):
            try :
                image_in_numbers = cv2.imread(os.path.join(path, image), cv2.IMREAD_COLOR)
                new_array = cv2.resize(image_in_numbers, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, label_to_number])
            except Exception as e:
                pass

create_training_data()

random.shuffle(training_data)

X = [] #Images (features)
y = [] #Breeds (labels)

for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
le=LabelEncoder()
y=le.fit_transform(y)
y=to_categorical(y,120)
X=X/255

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(120)) #The nodes equal to number of classes(breeds)
model.add(Activation("softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
history = model.fit(X, y, batch_size=64, epochs=40, validation_split=0.1)

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K
from keras.layers.normalization import BatchNormalization

from keras.applications.densenet import DenseNet121, preprocess_input
model2 = DenseNet121(input_shape=(224,224,3), pooling='avg',
                       weights="imagenet",
                       include_top=False)

x = model2.output
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

predictions = Dense(120, activation="softmax")(x) #The nodes equal to number of classes(breeds)

model_deepnet = Model(inputs = model2.input, outputs = predictions)

In [ ]:
model_deepnet.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
history2 = model_deepnet.fit(X, y, batch_size=16, epochs=20, validation_split=0.1)

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input
model3 = ResNet50(input_shape=(224,224,3), pooling='avg',
                       weights="imagenet",
                       include_top=False)

x = model3.output
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

predictions = Dense(120, activation="softmax")(x) #The nodes equal to number of classes(breeds)

model_resnet = Model(inputs = model3.input, outputs = predictions)

In [ ]:
model_resnet.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
history3 = model_resnet.fit(X, y, batch_size=16, epochs=20, validation_split=0.1)

In [ ]:
def download_and_predict(url, filename):
    # download and save
    os.system("curl -s {} -o {}".format(url, filename))
    img = Image.open(filename)
    img = img.convert('RGB')
    img = img.resize((224, 224))
    img.save(filename)
    # show image
    plt.figure(figsize=(4, 4))
    plt.imshow(img)
    plt.axis('off')
    # predict
    img = imread(filename)
    img = preprocess_input(img)
    probs = model2.predict(np.expand_dims(img, axis=0))
    for idx in probs.argsort()[0][::-1][:5]:
        print("{:.2f}%".format(probs[0][idx]*100), "\t", label_maps_rev[idx].split("-")[-1])

In [ ]:
download_and_predict("https://cdn.pixabay.com/photo/2016/07/25/00/06/corgi-1539598_1280.jpg",
                     "test_1.jpg")